In [1]:
import pandas as pd
import numpy as np
import datetime

In [2]:
train = pd.read_csv('./output/train.csv')    
test = pd.read_csv('./output/test.csv')    
test2 = pd.read_csv('./output/test2.csv')


In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 814262 entries, 0 to 814261
Data columns (total 13 columns):
hash             814262 non-null object
trajectory_id    814262 non-null object
time_entry       814262 non-null int64
time_exit        814262 non-null int64
vmax             814262 non-null float64
vmin             814262 non-null float64
vmean            814262 non-null float64
x_entry          814262 non-null float64
y_entry          814262 non-null float64
x_exit           814262 non-null float64
y_exit           814262 non-null float64
x_req            814262 non-null float64
y_req            814262 non-null float64
dtypes: float64(9), int64(2), object(2)
memory usage: 80.8+ MB


In [4]:
test.head()

,hash,trajectory_id,time_entry,time_exit,vmax,vmin,vmean,x_entry,y_entry,x_exit,y_exit,x_req,y_req
0,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_0,42197,42617,0.0,0.0,0.0,3.773413e+06,-1.909828e+07,3.773111e+06,-1.914508e+07,3.773413e+06,-1.909828e+07
1,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_2,44497,44497,0.0,0.0,0.0,3.773199e+06,-1.914354e+07,3.773199e+06,-1.914354e+07,3.773413e+06,-1.909828e+07
2,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_3,45267,47651,0.0,0.0,0.0,3.763760e+06,-1.921342e+07,3.771757e+06,-1.911092e+07,3.773199e+06,-1.914354e+07
3,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_4,48333,49393,0.0,0.0,0.0,3.773385e+06,-1.911344e+07,3.773131e+06,-1.914465e+07,3.763760e+06,-1.921342e+07
4,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_5,54212,54632,0.0,0.0,0.0,3.773118e+06,-1.914490e+07,NaN,NaN,3.773385e+06,-1.911344e+07


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

x_train = train.drop(columns=['hash','trajectory_id','x_exit','y_exit'])
x_test = test2.drop(columns=['hash','trajectory_id','x_exit','y_exit'])
y_train=pd.DataFrame()
y_train=train[['x_exit','y_exit']]


model = GradientBoostingRegressor(n_estimators=1000, 
                                  learning_rate=0.01,
                                  max_depth=5,
                                  #min_samples_split=3 ,#2
                                  #min_samples_leaf=2 ,#1 
                                  random_state=1234, 
                                  loss='ls',
                                 verbose=True,
                                 )

model.fit(x_train, y_train.x_exit.values)
predx = model.predict(x_test)

      Iter       Train Loss   Remaining Time 
         1    77974247.6138           31.93m
         2    76685516.4064           31.92m
         3    75422158.0828           32.10m
         4    74184102.6512           32.33m
         5    72970594.2865           32.42m
         6    71781279.6474           32.49m
         7    70615550.8951           33.12m
         8    69473062.8871           32.94m
         9    68353310.3864           32.78m
        10    67255712.2725           32.80m
        20    57409702.1471           32.13m
        30    49351960.3845           31.93m
        40    42756562.2110           31.28m
        50    37357429.7348           32.44m
        60    32937190.2795           32.85m
        70    29317234.4698           32.19m
        80    26343722.3504           31.48m
        90    23903887.6121           30.84m
       100    21899769.4187           30.33m
       200    13961752.8400           26.20m
       300    12698554.6213           22.77m
       40

In [ ]:
model.fit(x_train, y_train.y_exit.values)
predy = model.predict(x_test)

In [ ]:

#pred = pd.DataFrame(data=pred,columns=['x_exit','y_exit'])
pred = pd.DataFrame({'x_exit':predx,'y_exit':predy})

In [ ]:
def within_measure(x, y):
    #  3750901.5068 ≤ 𝑥 ≤ 3770901.5068
    #  −19268905.6133 ≤ 𝑦 ≤ −19208905.6133
    if 3750901.5068 <= x and x <= 3770901.5068 and -19268905.6133 <= y and y <= -19208905.6133:
        return 1
    else:
        return 0

X = pred['x_exit'].values 
Y = pred['y_exit'].values

city = [within_measure(x, y) for x,y in zip(X,Y)]

In [ ]:
submission= pd.DataFrame()
trajectory_id = test2['trajectory_id']
submission['id']=trajectory_id 
submission['target'] = city
submission.to_csv('./output/submission_gradientboosting.csv',index=False)    
submission[:20]

In [ ]:
min(city)

In [ ]:
max(city)